In [1]:
#https://towardsdatascience.com/lstm-by-example-using-tensorflow-feb0c1968537
#https://github.com/roatienza/Deep-Learning-Experiments/blob/master/Experiments/Tensorflow/RNN/rnn_words.py
#https://github.com/roatienza/Deep-Learning-Experiments/blob/master/Experiments/Tensorflow/RNN/belling_the_cat.txt
import numpy as np
import tensorflow as tf
from tensorflow.contrib import rnn
import random
import collections
import time

c:\users\mbolanos\appdata\local\conda\conda\envs\pabase\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Text file containing words for training
training_file = 'input_data.txt'

def read_data(fname):
    with open(fname) as f:
        content = f.readlines()
    content = [x.strip() for x in content]
    content = [content[i].split() for i in range(len(content))]
    content = np.array(content)
    content = np.reshape(content, [-1, ])
    return content

def build_dataset(words):
    count = collections.Counter(words).most_common()
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return dictionary, reverse_dictionary

def RNN(x, weights, biases):

    # reshape to [1, n_input]
    x = tf.reshape(x, [-1, n_input])

    # Generate a n_input-element sequence of inputs
    # (eg. [had] [a] [general] -> [20] [6] [33])
    x = tf.split(x,n_input,1)

    # 2-layer LSTM, each layer has n_hidden units.
    # Average Accuracy= 95.20% at 50k iter
    rnn_cell = rnn.MultiRNNCell([rnn.BasicLSTMCell(n_hidden),rnn.BasicLSTMCell(n_hidden)])

    # 1-layer LSTM with n_hidden units but with lower accuracy.
    # Average Accuracy= 90.60% 50k iter
    # Uncomment line below to test but comment out the 2-layer rnn.MultiRNNCell above
    # rnn_cell = rnn.BasicLSTMCell(n_hidden)

    # generate prediction
    outputs, states = rnn.static_rnn(rnn_cell, x, dtype=tf.float32)

    # there are n_input outputs but
    # we only want the last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

def elapsed(sec):
    if sec<60:
        return str(sec) + " sec"
    elif sec<(60*60):
        return str(sec/60) + " min"
    else:
        return str(sec/(60*60)) + " hr"

In [3]:
training_data = read_data(training_file)

In [4]:
dictionary, reverse_dictionary = build_dataset(training_data)
vocab_size = len(dictionary)

In [5]:
# Target log path
logs_path = 'C:/MYLOCALFILES/JUPYTER_NOTEBOOKS/DEEP_LEARNING_OTHERS/LSTM_TF_SAMPLE_1/rnn_words/'
writer = tf.summary.FileWriter(logs_path)

# Parameters
learning_rate = 0.001
training_iters = 50000
display_step = 1000
n_input = 3

# number of units in RNN cell
n_hidden = 512

# tf Graph input
x = tf.placeholder("float", [None, n_input, 1])
y = tf.placeholder("float", [None, vocab_size])

# RNN output node weights and biases
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, vocab_size]))
}
biases = {
    'out': tf.Variable(tf.random_normal([vocab_size]))
}

In [6]:
start_time = time.time()

pred = RNN(x, weights, biases)

# Loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(cost)

# Model evaluation
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

# Launch the graph
saver = tf.train.Saver()
with tf.Session() as session:
    session.run(init)
    step = 0
    offset = random.randint(0,n_input+1)
    end_offset = n_input + 1
    acc_total = 0
    loss_total = 0

    writer.add_graph(session.graph)

    while step < training_iters:
        # Generate a minibatch. Add some randomness on selection process.
        if offset > (len(training_data)-end_offset):
            offset = random.randint(0, n_input+1)

        symbols_in_keys = [ [dictionary[ str(training_data[i])]] for i in range(offset, offset+n_input) ]
        symbols_in_keys = np.reshape(np.array(symbols_in_keys), [-1, n_input, 1])

        symbols_out_onehot = np.zeros([vocab_size], dtype=float)
        symbols_out_onehot[dictionary[str(training_data[offset+n_input])]] = 1.0
        symbols_out_onehot = np.reshape(symbols_out_onehot,[1,-1])

        _, acc, loss, onehot_pred = session.run([optimizer, accuracy, cost, pred], \
                                                feed_dict={x: symbols_in_keys, y: symbols_out_onehot})
        loss_total += loss
        acc_total += acc
        if (step+1) % display_step == 0:
            print("Iter= " + str(step+1) + ", Average Loss= " + \
                  "{:.6f}".format(loss_total/display_step) + ", Average Accuracy= " + \
                  "{:.2f}%".format(100*acc_total/display_step))
            acc_total = 0
            loss_total = 0
            symbols_in = [training_data[i] for i in range(offset, offset + n_input)]
            symbols_out = training_data[offset + n_input]
            symbols_out_pred = reverse_dictionary[int(tf.argmax(onehot_pred, 1).eval())]
            print("%s - [%s] vs [%s]" % (symbols_in,symbols_out,symbols_out_pred))
        step += 1
        offset += (n_input+1)
    
    print("Optimization Finished!")
    
    save_path = saver.save(session, "./model.ckpt")
    print("Model Saved..")
    
    print("Elapsed time: ", elapsed(time.time() - start_time))
    print("Run on command line.")
    print("\ttensorboard --logdir=%s" % (logs_path))
    print("Point your web browser to: http://localhost:6006/")
    if 0==1:
        while True:
            prompt = "%s words: " % n_input
            sentence = input(prompt)
            sentence = sentence.strip()
            words = sentence.split(' ')
            if len(words) != n_input:
                continue
            try:
                symbols_in_keys = [dictionary[str(words[i])] for i in range(len(words))]
                for i in range(32):
                    keys = np.reshape(np.array(symbols_in_keys), [-1, n_input, 1])
                    onehot_pred = session.run(pred, feed_dict={x: keys})
                    onehot_pred_index = int(tf.argmax(onehot_pred, 1).eval())
                    sentence = "%s %s" % (sentence,reverse_dictionary[onehot_pred_index])
                    symbols_in_keys = symbols_in_keys[1:]
                    symbols_in_keys.append(onehot_pred_index)
                print(sentence)
            except:
                print("Word not in dictionary")

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

Iter= 1000, Average Loss= 4.475737, Average Accuracy= 2.60%
['nobody', 'spoke', '.'] - [then] vs [then]
Iter= 2000, Average Loss= 2.695944, Average Accuracy= 24.50%
['got', 'up', 'and'] - [said] vs [neighbourhood]
Iter= 3000, Average Loss= 2.404791, Average Accuracy= 34.60%
['this', 'proposal', 'met'] - [with] vs [well]
Iter= 4000, Average Loss= 2.162242, Average Accuracy= 43.00%
['in', 'the', 'neighbourhood'] - [.] vs [our]
Iter= 5000, Average Loss= 1.988678, Average Accuracy= 48.70%
['we', 'should', 'always'] - [know] vs [know]
Iter= 6000, Average Loss= 1.749640, Average Accuracy= 56.50%
['the', 'cat', '.'] - [by] vs [some]
Iter= 7000, Average Loss= 1.572564, Average Accuracy= 60.70%
['procured', ',', 'and'] - [attached] vs [could]
Iter= 8000, Average Loss= 1.300953, Average Accuracy= 65.50%
['of',

In [15]:
#test prediction
with tf.Session() as session:
    saver.restore(session, "./model.ckpt")
    offset=5
    print([ [ str(training_data[i])] for i in range(offset, offset+n_input) ])
    symbols_in_keys = [ [dictionary[ str(training_data[i])]] for i in range(offset, offset+n_input) ]
    symbols_in_keys = np.reshape(np.array(symbols_in_keys), [-1, n_input, 1])

    symbols_out_onehot = np.zeros([vocab_size], dtype=float)
    symbols_out_onehot[dictionary[str(training_data[offset+n_input])]] = 1.0
    symbols_out_onehot = np.reshape(symbols_out_onehot,[1,-1])

    _, acc, loss, onehot_pred = session.run([optimizer, accuracy, cost, pred], \
                                            feed_dict={x: symbols_in_keys, y: symbols_out_onehot})
    
    symbols_out_pred = reverse_dictionary[int(tf.argmax(onehot_pred, 1).eval())]
    print(symbols_out_pred)

INFO:tensorflow:Restoring parameters from ./model.ckpt
[[','], ['the'], ['cat']]
.


In [49]:
offset=20
keys = [ [dictionary[ str(training_data[i])]] for i in range(offset, offset+n_input) ]

result_keys = []
result_keys.extend(keys)
step = 1

with tf.Session() as session:
    saver.restore(session, "./model.ckpt")
    
    #PREDICTIONS
    while step<10:
        symbols_in_keys = keys
        symbols_in_keys = np.reshape(np.array(symbols_in_keys), [-1, n_input, 1])

        onehot_pred = session.run(pred, feed_dict={x: symbols_in_keys})
        onehot_pred_index = int(tf.argmax(onehot_pred, 1).eval())

        keys.append([onehot_pred_index]) ##for next prediction
        result_keys.append([onehot_pred_index]) #for all prediction

        ##remove first item
        del keys[0]

        #increment steps
        step+=1

#Show Result
[reverse_dictionary[k[0]]   for k in result_keys]

INFO:tensorflow:Restoring parameters from ./model.ckpt


['enemy',
 ',',
 'the',
 'cat',
 '.',
 'by',
 'this',
 'means',
 'we',
 'should',
 'always',
 'know']